In [1]:
import pandas as pd
from sqlalchemy import create_engine, Column, Integer, String, MetaData, Table
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy.util import deprecations
from sqlalchemy.sql import text
deprecations.SILENCE_UBER_WARNING = True
# create database connection
import psycopg2
import psycopg2.extras
from typing import Iterator, Dict, Any

def insert_execute_batch(ssql, df_dict) -> None:
    
    connection = psycopg2.connect(
        host="98.98.117.105",
        port='5432',
        database="medols",
        user="postgres",
        password='FEWcTB3JIX5gK4T06c1MdkM9N2S8w9pb',
    )

    try:
        connection.autocommit = True
        with connection.cursor() as cursor:
            psycopg2.extras.execute_batch(cursor, ssql, df_dict)

    except:
        # Rollback changes if there's an error
        print(f"Error executing query: {e}")

    finally:
        connection.close()

def execute_query_psql(query, params=None):
    # Set your PostgreSQL connection parameters
    db_params = {
        'host': '98.98.117.105',
        'port': '5432',
        'database': 'medols',
        'user': 'postgres',
        'password': 'FEWcTB3JIX5gK4T06c1MdkM9N2S8w9pb',
    }

    # Create a SQLAlchemy engine
    engine = create_engine(f"postgresql+psycopg2://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['database']}")

    # Create a metadata object
    metadata = MetaData()

    # Create a session
    Session = sessionmaker(bind=engine)
    session = Session()

    try:
        # Execute the query with optional parameters
        result = session.execute(text(query), params)

        # Check if the query is a SELECT query
        is_select_query = result.returns_rows

        if is_select_query:
            # Fetch the data and return as a Pandas DataFrame
            columns = result.keys()
            fetched_data = result.fetchall()
            df = pd.DataFrame(fetched_data, columns=columns)
            # print("Fetched Data as DataFrame:")
            # print(df)
            return df
        else:
            # Get the number of rows affected for non-SELECT queries
            rows_affected = result.rowcount

            # Commit the changes to the database for non-SELECT queries
            session.commit()

            # print(f"Query executed successfully. {rows_affected} rows affected.")
            return rows_affected
    except Exception as e:
        # Rollback changes if there's an error
        session.rollback()
        print(f"Error executing query: {e}")
    finally:
        # Close the session
        session.close()


In [2]:
from datetime import datetime
import re
import logging

# Configure logging
# logging.basicConfig(filename='/home/jup_user/multipool/ai-case-study/date_parsing_errors.log', level=logging.ERROR, format='%(asctime)s:%(levelname)s:%(message)s')

from datetime import datetime

# Updated dictionary to map Indonesian month abbreviations and full names to English equivalents
month_translation = {
    "Jan": "January", "Januari": "January",
    "Feb": "February", "Februari": "February",
    "Mar": "March", "Maret": "March",
    "Apr": "April",
    "Mei": "May",
    "Jun": "June", "Juni": "June",
    "Jul": "July", "Juli": "July",
    "Agu": "August", "Agustus": "August",
    "Sep": "September",
    "Okt": "October",
    "Nov": "November",
    "Des": "December"
}

def convert_to_datetime3(date_str):
    # Remove any day of the week, "WIB", and time information (hours:minutes)
    date_str = re.sub(r"(Senin|Selasa|Rabu|Kamis|Jumat|Sabtu|Minggu)", "", date_str)  # Remove day of the week
    date_str = re.sub(r"- \d{2}:\d{2} WIB", "", date_str)  # Remove time and "WIB"
    date_str = re.sub(r"Jam \d{2}:\d{2}", "", date_str)  # Remove time information (another format)
    
    # Strip extra spaces after removing parts of the string
    date_str = date_str.strip()
    
    # Handle different formats
    if "," in date_str:  # Format: "Agu 14, 2024" or "Senin, 26 Agustus 2024 Jam 21:12"
        date_part = date_str.split()[:3]  # Take the first three components for date (e.g., month, day, year)
        # Remove the comma from the day
        date_part[1] = date_part[1].replace(",", "")
        month_str = date_part[0]
    else:  # Format: "26 Agustus 2024" or "26 August 2024"
        date_part = date_str.split()  # Split by space to separate day, month, year
        month_str = date_part[1]
    
    # Translate the month name from Indonesian to English
    english_month = month_translation.get(month_str)
    
    if not english_month:
        # If month_str is already in English (e.g., "August"), use it directly
        english_month = month_str
    
    # Create a date string in a standard format
    standard_date_str = f"{english_month} {date_part[0]}, {date_part[2]}"
    
    # Convert the standard date string to a datetime object
    return datetime.strptime(standard_date_str, "%B %d, %Y")



def convert_to_python_datetime(date_str):
    # Mapping Indonesian day names to English
    days_ind_to_eng = {
        'Senin': 'Monday',
        'Selasa': 'Tuesday',
        'Rabu': 'Wednesday',
        'Kamis': 'Thursday',
        'Jumat': 'Friday',
        'Sabtu': 'Saturday',
        'Minggu': 'Sunday'
    }

    # Indonesian month names to English
    months_ind_to_eng = {
        'Januari': 'January',
        'Februari': 'February',
        'Maret': 'March',
        'April': 'April',
        'Mei': 'May',
        'Juni': 'June',
        'Juli': 'July',
        'Agustus': 'August',
        'September': 'September',
        'Oktober': 'October',
        'November': 'November',
        'Desember': 'December'
    }

    try:
        print("Original Date: %s" % (date_str))
        date_str = date_str.replace("Jum'at", "Jumat")

        # Split the date string and trim whitespace around elements
        parts = [part.strip() for part in date_str.split(',')]
        day_part = parts[0].split()[0]  # Split "Kamis," and take "Kamis"
        date_and_month = parts[1].split()  # Split "02 Mei  2024 06:01 WIB"
        
        # Replace Indonesian day and month names with English equivalents
        day_eng = days_ind_to_eng[day_part]
        month_ind = date_and_month[1]
        month_eng = months_ind_to_eng[month_ind]
        print("Replaced: %s" % (date_and_month))
        
        # check for correct pattern
        if (date_and_month[3] != '-'):
            # Construct the cleaned and translated date string
            cleaned_date_str = f"{day_eng}, {date_and_month[0]} {month_eng} {date_and_month[2]} {date_and_month[3]} WIB"

        else:
            # Construct the cleaned and translated date string
            cleaned_date_str = f"{day_eng}, {date_and_month[0]} {month_eng} {date_and_month[2]} {date_and_month[4]} WIB"
        
        print("Cleaned : %s" % (cleaned_date_str))
        
        # Define the format of the incoming date string
        date_format = '%A, %d %B %Y %H:%M WIB'
    
        # Parse the date string into a datetime object
        date_obj = datetime.strptime(cleaned_date_str, date_format)

        return date_obj

    except Exception as e:
        # print(e)
        return None
            
def parse_date_to_datetime(date_str):
    # Regex pattern to extract the date, month, year, and time from the string
    pattern = r"\w+\s+(\d{1,2})\s+(\w+)\s+(\d{4})\s+-\s+(\d{2}:\d{2})\s+WIB"
    match = re.search(pattern, date_str)

    try:
        if match:
            day, month, year, time = match.groups()
            day, year = int(day), int(year)
    
            # Map months from Bahasa Indonesia to English for datetime compatibility
            month_mapping = {
                'Januari': 1, 'Februari': 2, 'Maret': 3, 'April': 4, 'Mei': 5, 'Juni': 6,
                'Juli': 7, 'Agustus': 8, 'September': 9, 'Oktober': 10, 'November': 11, 'Desember': 12
            }
            
            month = month_mapping.get(month, None)
            if month is None:
                return f"Error: Month '{month}' is not recognized."
    
            # Convert date and time into a datetime object
            hour, minute = map(int, time.split(':'))
            date_time = datetime(year, month, day, hour, minute)
    
            return date_time
        else:
            # print("parse_date_to_datetime %s" % date_str)
            return None

    except Exception as e:
        logging.error(f"Original Data: {date_str}, ID: {row.get('id', 'Unknown ID')}")
        return None

def parse_bahasa_date_enhanced(row):
    # Extended month mappings including abbreviations for Bahasa Indonesia and English
    month_mapping_enhanced = {
        'Januari': 1, 'Jan': 1, 'January': 1,
        'Februari': 2, 'Feb': 2, 'February': 2,
        'Maret': 3, 'Mar': 3, 'March': 3,
        'April': 4, 'Apr': 4,
        'Mei': 5, 'May': 5,
        'Juni': 6, 'Jun': 6, 'June': 6,
        'Juli': 7, 'Jul': 7, 'July': 7,
        'Agustus': 8, 'Ags': 8, 'Aug': 8, 'August': 8, 'Agu':8,
        'September': 9, 'Sep': 9,
        'Oktober': 10, 'Okt': 10, 'Oct': 10,
        'November': 11, 'Nov': 11,
        'Desember': 12, 'Des': 12, 'Dec': 12, 'December': 12
    }
    date_str = row['publish_date']
    # Clean up the string by removing vertical bars and commas
    date_str = date_str.replace('|', ' ').replace(',', ' ').strip()
    # Replace "Jum'at" with "Jumat"
    date_str = date_str.replace("Jum'at", "Jumat")
    print("converted month %s" % (date_str))

    try:
        # Regex patterns to capture different date formats
        match = re.search(r"(\d{1,2})[\s/-](\d{1,2}|[a-zA-Z]+)[\s/-](\d{4})\s+(\d{2}):(\d{2})", date_str)
        if match:
            day, month, year, hour, minute = match.groups()
            if month.isdigit():
                month = int(month)
            else:
                month = month_mapping_enhanced.get(month.capitalize())
                if month is None:
                    raise ValueError(f"Unknown month: {month}")

            # Create datetime object and format it
            dt = datetime(int(year), month, int(day), int(hour), int(minute))
            # print(dt)
            return dt.strftime('%Y-%m-%d %H:%M:%S'), row['id']
        else:
            # further combination
            test1 = convert_to_datetime3(date_str)
            test2 = convert_to_python_datetime(date_str)
            test3 = parse_date_to_datetime(date_str)
            if test1 is not None:
                print("test1 %s" % test1)
                return test1, row['id']
            elif test2 is not None:
                print("test2 %s" % test2)
                return test2, row['id']
            elif test3 is not None:
                print("test3 %s" % test3)
                return test3, row['id']
                
    except Exception as e:
        logging.error(f"Original Data: {date_str}, ID: {row.get('id', 'Unknown ID')}")
        print(f"Error Msg: {e}, Original Data: {date_str}, ID: {row.get('id', 'Unknown ID')}")
        return None


In [3]:
def update_data_sql(row):
    if row is None:
        print(row)
        
    elif row[0] is None:
        ssql = """
        update news_national_medol 
        set    calculation_mark = 1
        where id = %s        
        """ % (row[1])
        execute_query_psql(ssql,)

    else:    
        ssql = """
        update news_national_medol 
        set    publish_date_convert = '%s', 
               calculation_mark = 1
        where id = %s
        """ % (row[0],row[1])
        execute_query_psql(ssql,)

In [4]:
# select * from news_national_medol where calculation_mark is null
sql = """
select * from news_national_medol where calculation_mark is null
"""
df_source = execute_query_psql(sql)
if len(df_source) > 0:
    df_res = df_source.apply(parse_bahasa_date_enhanced, axis=1)
    df_res.apply(update_data_sql)

ERROR:root:Original Data: , ID: 106376
ERROR:root:Original Data: , ID: 106377
ERROR:root:Original Data: , ID: 106378
ERROR:root:Original Data: , ID: 106399
ERROR:root:Original Data: , ID: 106470
ERROR:root:Original Data: , ID: 106471
ERROR:root:Original Data: , ID: 106472
ERROR:root:Original Data: , ID: 106475
ERROR:root:Original Data: , ID: 106476
ERROR:root:Original Data: , ID: 106477
ERROR:root:Original Data: , ID: 106478
ERROR:root:Original Data: , ID: 106479
ERROR:root:Original Data: , ID: 106480
ERROR:root:Original Data: , ID: 106481
ERROR:root:Original Data: , ID: 106482
ERROR:root:Original Data: , ID: 106483
ERROR:root:Original Data: , ID: 106484
ERROR:root:Original Data: , ID: 106485
ERROR:root:Original Data: , ID: 106486
ERROR:root:Original Data: , ID: 106487
ERROR:root:Original Data: , ID: 106488
ERROR:root:Original Data: , ID: 106489
ERROR:root:Original Data: , ID: 106490
ERROR:root:Original Data: , ID: 106491
ERROR:root:Original Data: , ID: 106492
ERROR:root:Original Data:

converted month 
Error Msg: list index out of range, Original Data: , ID: 106376
converted month 
Error Msg: list index out of range, Original Data: , ID: 106377
converted month 
Error Msg: list index out of range, Original Data: , ID: 106378
converted month 
Error Msg: list index out of range, Original Data: , ID: 106399
converted month 
Error Msg: list index out of range, Original Data: , ID: 106470
converted month 
Error Msg: list index out of range, Original Data: , ID: 106471
converted month 
Error Msg: list index out of range, Original Data: , ID: 106472
converted month 
Error Msg: list index out of range, Original Data: , ID: 106475
converted month 
Error Msg: list index out of range, Original Data: , ID: 106476
converted month 
Error Msg: list index out of range, Original Data: , ID: 106477
converted month 
Error Msg: list index out of range, Original Data: , ID: 106478
converted month 
Error Msg: list index out of range, Original Data: , ID: 106479
converted month 
Error Msg: 

In [5]:
def update_data_sql_tbn(row):
    if row is not None:
        ssql = """
        update news_tbn_medol 
        set    converted_date = '%s'
        where id = %s
        """ % (row[0],row[1])
        print(ssql)
        execute_query_psql(ssql,)
        
# select * from news_tbn_medol where converted_date is null
sql = """
select * from news_tbn_medol where converted_date is null
"""
df_source = execute_query_psql(sql)
if len(df_source) > 0:
    df_res = df_source.apply(parse_bahasa_date_enhanced, axis=1)
    df_res.apply(update_data_sql_tbn)

ERROR:root:Original Data: , ID: 4467
ERROR:root:Original Data: 27/08/2024, ID: 15696
ERROR:root:Original Data: 27/08/2024, ID: 15697
ERROR:root:Original Data: 27/08/2024, ID: 15698
ERROR:root:Original Data: 27/08/2024, ID: 15699
ERROR:root:Original Data: 27/08/2024, ID: 15700
ERROR:root:Original Data: 27/08/2024, ID: 15701
ERROR:root:Original Data: 27/08/2024, ID: 15702
ERROR:root:Original Data: , ID: 6835
ERROR:root:Original Data: , ID: 6836
ERROR:root:Original Data: 27/08/2024, ID: 15703
ERROR:root:Original Data: 27/08/2024, ID: 15704
ERROR:root:Original Data: August 27  2024, ID: 15710
ERROR:root:Original Data: August 27  2024, ID: 15711
ERROR:root:Original Data: August 27  2024, ID: 15712
ERROR:root:Original Data: August 27  2024, ID: 15713
ERROR:root:Original Data: August 27  2024, ID: 15714
ERROR:root:Original Data: August 27  2024, ID: 15715
ERROR:root:Original Data: August 27  2024, ID: 15716
ERROR:root:Original Data: August 27  2024, ID: 15717
ERROR:root:Original Data: August 

converted month 28 August 2024
Original Date: 28 August 2024
test1 2024-08-28 00:00:00
converted month 28 August 2024
Original Date: 28 August 2024
test1 2024-08-28 00:00:00
converted month 28 August 2024
Original Date: 28 August 2024
test1 2024-08-28 00:00:00
converted month 28 August 2024
Original Date: 28 August 2024
test1 2024-08-28 00:00:00
converted month 
Error Msg: list index out of range, Original Data: , ID: 4467
converted month 28 August 2024
Original Date: 28 August 2024
test1 2024-08-28 00:00:00
converted month 28 August 2024
Original Date: 28 August 2024
test1 2024-08-28 00:00:00
converted month 28 August 2024
Original Date: 28 August 2024
test1 2024-08-28 00:00:00
converted month 28 August 2024
Original Date: 28 August 2024
test1 2024-08-28 00:00:00
converted month 28/08/2024 07:00:00
converted month 28/08/2024 07:20:00
converted month 28/08/2024 07:40:00
converted month 28/08/2024 08:00:00
converted month 28/08/2024 08:20:00
converted month 28/08/2024 08:40:00
converted